# Section 0

QPM: Assignment 5

DENG Yunyun <br>
LI Jiaxin <br>
SBAI Ilyas <br>
ZHOU Zhichen <br>

Note: NA

# Initial step to prepare the data

Access data stored in the database

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import sqlite3

#Establish the connection to the SQLite database
SnP500 = sqlite3.connect(database="/Users/lijiaxin/Downloads/M2/S2/QPM/snp500.sqlite")

log_return=(pd.read_sql_query(
        sql="SELECT * FROM TenCompanies_log_ret_monthly",
        con=SnP500))

log_return.set_index('date', inplace=True)
log_return.index=pd.to_datetime(log_return.index)
log_return

,MMM,AOS,ABT,ADM,ADBE,ADP,AES,AFL,A,AKAM
date,,,,,,,,,,
2000-02-29,-0.053376,-0.135342,0.013347,-0.150862,0.616504,-0.085216,0.044994,-0.170378,0.450696,0.047523
2000-03-31,0.004243,0.042560,0.064183,0.024541,0.087614,0.104224,-0.062304,0.220062,0.001203,-0.485239
2000-04-30,-0.022120,0.139391,0.093347,-0.037041,0.082932,0.109110,0.132836,0.068901,-0.159977,-0.486383
2000-05-31,-0.003692,0.021054,0.056887,0.188229,-0.071765,0.021828,-0.030337,0.058972,-0.185430,-0.392902
2000-06-30,-0.032595,-0.003728,0.090971,-0.196027,0.144227,-0.024957,0.044825,-0.117935,0.001697,0.575935
...,...,...,...,...,...,...,...,...,...,...
2022-08-31,-0.131211,-0.114056,-0.058554,0.064676,-0.093692,0.013553,0.135723,0.042781,-0.044604,-0.063721
2022-09-30,-0.118085,-0.150143,-0.059091,-0.088450,-0.305259,-0.073160,-0.118766,-0.055714,-0.053656,-0.116897
2022-10-31,0.129598,0.125910,0.026987,0.186869,0.146124,0.066322,0.152271,0.147161,0.131190,0.095061


In [25]:
#check if the result matches with solution for A3
log_return[log_return.index=='2014-10-31']

,MMM,AOS,ABT,ADM,ADBE,ADP,AES,AFL,A,AKAM
date,,,,,,,,,,
2014-10-31,0.081887,0.123641,0.052267,-0.083637,0.013352,0.11438,-0.004052,0.02509,-0.030289,0.008326


# Section 1

We first define the necessary data: the estimation window ```T_est```, the vector of monthly mean returns ```mu```, the variance-covariance matrix of annualised returns ```V``` and its inverse ```Vinverse```. <br>
We use the data from the first 60 days.

In [2]:
from scipy.optimize import minimize

In [12]:
#Set the first 60 months to be the estimation window
T_est = 60
sp = log_return.iloc[0:T_est,:]

#risk free rate is set to 0
Rf = 0

# Investor’s risk aversion
# The value chosen matches that for risk-minimization formulation
#gamma=2.715745
gamma = 2

#Define a column vector of ones
n=len(log_return.columns)
ones = np.ones((n,1))

#Column vector of monthly mean returns
mu = sp.mean().to_numpy().reshape(-1,1)

#Variance-covariance matrix
V = sp.cov().to_numpy()

#Define inverse of the variance-covariance matrix
Vinverse = np.linalg.inv(V)

In [13]:
# Define the objective function to be minimized
# Note that the optimizer only minimizes,
# so reverse sign of objective if it is to be maximized

def objective_MVP(weights):
    portfolio_return = weights.T @ (mu - Rf * ones)
    portfolio_risk = weights.T @ V @ weights
    return (gamma/2) * portfolio_risk - portfolio_return

# Define the initial guess for weights
initial_weights = np.ones(n)/n

# Define the equality constraint (weights sum to 1)
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) -1})

# Add a lower-bound constraint of 0 to the above problem
# There is no upper bound, so I have set it to 999

bounds = tuple((0, 999) for _ in range (n))

# When calling minimize, we add ‘‘bounds’’ to the minimization function
result = minimize(objective_MVP, initial_weights, method='SLSQP', bounds = bounds, constraints=constraints)

In [14]:
# Extract the optimal weights from results
optimal_weights = result.x

#Weights in the risk-free asset
w_riskFree = 1 - ones.T @ optimal_weights

#Weights of the ‘‘tangency’’ portfolio
w_tangency = optimal_weights / (ones.T @ optimal_weights)

w_mvp_shortsaleConstrained = list(map(lambda num: format(num, '.6f'), optimal_weights))
w_tangency_formatted = list(map(lambda num: format(num, '.6f'), w_tangency))

print("\n* Using numerical solver and with short-sale constraints")
print("\nWeights of the mean-variance portfolio (MVP) are = ")
print(w_mvp_shortsaleConstrained)
print("\nWeight in the risk-free asset = ")
print(w_riskFree)
print("\nWeights of the tangency portfolio (based on MVP) are = ")
print(w_tangency_formatted)


* Using numerical solver and with short-sale constraints

Weights of the mean-variance portfolio (MVP) are = 
['0.299701', '0.000000', '0.000000', '0.504113', '0.128858', '0.000000', '0.000000', '0.067328', '0.000000', '0.000000']

Weight in the risk-free asset = 
[0.]

Weights of the tangency portfolio (based on MVP) are = 
['0.299701', '0.000000', '0.000000', '0.504113', '0.128858', '0.000000', '0.000000', '0.067328', '0.000000', '0.000000']


Solution for GMV with short-sale constraints

In [15]:
# Define the objective function to be minimized (portfolio variance)

def objective_GMV(weights, V):
    portfolio_variance = weights.T @ V @ weights
    return portfolio_variance

# Define the equality constraint (weights sum to 1)
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) -1})

# Use scipy.optimize.minimize to find the optimal weights
result = minimize(objective_GMV, initial_weights, args=(V,), method='SLSQP',
bounds=bounds, constraints=constraints)

# Extract the optimal weights

optimal_weights = result.x

# Print the results
print("\n* Using numeric solution with short-sale constraints")
print("\nOptimal Weights:")
for i, weight in enumerate(optimal_weights):
    print(f"\nAsset {i + 1}: {weight:.6f}")
    print("Minimum Portfolio Variance is:", result.fun)


* Using numeric solution with short-sale constraints

Optimal Weights:

Asset 1: 0.297956
Minimum Portfolio Variance is: 0.001779839494870714

Asset 2: 0.091313
Minimum Portfolio Variance is: 0.001779839494870714

Asset 3: 0.294329
Minimum Portfolio Variance is: 0.001779839494870714

Asset 4: 0.064539
Minimum Portfolio Variance is: 0.001779839494870714

Asset 5: 0.025332
Minimum Portfolio Variance is: 0.001779839494870714

Asset 6: 0.084256
Minimum Portfolio Variance is: 0.001779839494870714

Asset 7: 0.003484
Minimum Portfolio Variance is: 0.001779839494870714

Asset 8: 0.138793
Minimum Portfolio Variance is: 0.001779839494870714

Asset 9: 0.000000
Minimum Portfolio Variance is: 0.001779839494870714

Asset 10: 0.000000
Minimum Portfolio Variance is: 0.001779839494870714


In [16]:
np.sum(optimal_weights)

1.0

# Section 2

We use the rolling window $T^{est}=60$ to estimate the time series of optimal weights for both strategies by looping through the dates.

In [17]:
#set the target return for mean variance portfolio

#create empty list to store the weights
w_MVP_list = []
w_GMV_list = []

Rf=0,
ones=np.ones((n,1))

#loop through the dates,
#at each date compute the weights for the two strategies based on the previous 60 days of monthly returns

for i in range (0,len(log_return)-60):
    #calculate the metrics for each window
    sp_est = log_return.iloc[i:T_est+i,:]
    mu = sp_est.mean().to_numpy().reshape(-1,1)
    V = sp_est.cov().to_numpy()
    Vinverse = np.linalg.inv(V)
    
    result_MVP = minimize(objective_MVP, initial_weights, method='SLSQP', bounds = bounds, constraints=constraints)
    w_MVP = result_MVP.x
    
    result_GMV = minimize(objective_GMV, initial_weights, args=(V,), method='SLSQP',
                          bounds=bounds, constraints=constraints) 
    w_GMV = result_GMV.x
    
    w_MVP_list.append(w_MVP)
    w_GMV_list.append(w_GMV)

#create the time series of weights by adding up the datetime index
w_MVP_ts = pd.Series(w_MVP_list, index=log_return.index[60:276])
w_GMV_ts = pd.Series(w_GMV_list, index=log_return.index[60:276])

In [18]:
w_MVP_ts

date
2005-02-28    [0.29970077771986314, 0.0, 0.0, 0.504113165854...
2005-03-31    [0.22337423031480635, 0.0, 0.0, 0.574837626042...
2005-04-30    [0.37566265142889244, 0.0033639329772872017, 0...
2005-05-31    [0.44096426397385613, 0.037492164630596805, 0....
2005-06-30    [0.5956816349361854, 0.09566697889349875, 0.0,...
                                    ...                        
2022-08-31    [0.0, 0.0, 0.36027598526227556, 0.160756641684...
2022-09-30    [0.0, 3.719117917270417e-18, 0.117677317787011...
2022-10-31    [4.319382790219587e-17, 1.094012396213501e-18,...
2022-11-30    [0.0, 0.0, 0.0, 0.515420836140725, 1.512290201...
2022-12-31    [0.0, 9.908107490823384e-17, 0.0, 0.3787326939...
Length: 215, dtype: object

In [19]:
w_GMV_ts

date
2005-02-28    [0.29795592767945606, 0.09131272736634102, 0.2...
2005-03-31    [0.2884157061326681, 0.09340644859051198, 0.27...
2005-04-30    [0.25716579098213943, 0.0919420886358533, 0.26...
2005-05-31    [0.22166645908553226, 0.11088265435555311, 0.2...
2005-06-30    [0.21419910559443592, 0.10570903108669749, 0.2...
                                    ...                        
2022-08-31    [0.026171989604653226, 1.0408340855860843e-17,...
2022-09-30    [0.0, 2.289235386371449e-18, 0.269705882257739...
2022-10-31    [1.0408340855860843e-17, 0.0, 0.25742448355670...
2022-11-30    [0.0, 0.0, 0.2855515937084144, 0.1332613675564...
2022-12-31    [0.017236729248999827, 0.0, 0.2822520663320782...
Length: 215, dtype: object

# Section 3

We calculate the out-of-sample returns for both strategies at time $t+1$ by multiplying the weights estimated from the information available until time $t$ by the stock returns at time $t+1$.

In [20]:
#create an empty list
ret_MVP_list = []
ret_GMV_list = []

for i in range (0,len(log_return)-60):
    ret_MVP_est = (log_return.iloc[T_est+i,:].to_numpy())@w_MVP_ts[i]
    ret_MVP_list.append(ret_MVP_est)
    
    ret_GMV_est = (log_return.iloc[T_est+i,:].to_numpy())@w_GMV_ts[i]
    ret_GMV_list.append(ret_GMV_est)
    
#create the time series of monthly returns by adding up the datetime index 
ret_MVP_ts = pd.Series(ret_MVP_list, index=log_return.index[60:])
ret_GMV_ts = pd.Series(ret_GMV_list, index=log_return.index[60:])

# Section 4

In [21]:
SR_MVP = np.sqrt(12)*np.mean(ret_MVP_ts) / np.std(ret_MVP_ts)
SR_GMV = np.sqrt(12)*np.mean(ret_GMV_ts) / np.std(ret_GMV_ts)

print('The Sharpe ratio for the mean-variance portfolio is:',SR_MVP,'\n')
print('The Sharpe ratio for the global minimum variance portfolio is:',SR_GMV, '\n')

if SR_MVP > SR_GMV:
    print('The mean-variance portfolio has higher sharpe ratio.')
else: 
    print('The global minimum variance portfolio has higher sharpe ratio.')

The Sharpe ratio for the mean-variance portfolio is: 0.4190485923425447 

The Sharpe ratio for the global minimum variance portfolio is: 0.7044580011322243 

The global minimum variance portfolio has higher sharpe ratio.
